# Telco Churn Analysis

## Name: Siddheshwari Sapra
####  EDA of the data

In [176]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN

In [177]:
## Loading libraries
!pip install imbalanced-learn

You should consider upgrading via the 'C:\Users\rishi\anaconda3\python.exe -m pip install --upgrade pip' command.


In [178]:
telco_base_data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [179]:
telco_base_data.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [180]:
telco_base_data = telco_base_data.drop(columns=['customerID', 'gender','PhoneService','MultipleLines', 'InternetService'], axis=1)

In [181]:
telco_base_data.columns

Index(['SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [182]:
telco_base_data

,SeniorCitizen,Partner,Dependents,tenure,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,Yes,No,1,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,0,No,No,34,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,0,No,No,2,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,0,No,No,45,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,0,No,No,2,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,Yes,Yes,24,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,0,Yes,Yes,72,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,0,Yes,Yes,11,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,1,Yes,No,4,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [194]:
X = telco_base_data.drop('Churn', axis=1)

In [195]:
y = telco_base_data['Churn']

In [196]:
# TotalCharges columns has numeric values but it was showing dtype of object. Therefore converting it to numeric
X['TotalCharges'] = pd.to_numeric(X['TotalCharges'], errors='coerce')

In [197]:
X.columns

Index(['SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges'],
      dtype='object')

In [198]:
# Convert all categorical columns to numerical categorical columns
cols = X.select_dtypes('object').columns
# cat_data = pd.get_dummies(X[cols])

In [199]:
from sklearn.preprocessing import OneHotEncoder

In [200]:
enc = OneHotEncoder(handle_unknown='ignore')

In [201]:
cat_data = enc.fit(telco_base_data[cols])
cat_data = pd.DataFrame(enc.transform(X[cols]).toarray())

In [219]:
cat_data.columns = enc.get_feature_names()

In [220]:
cat_data

,x0_No,x0_Yes,x1_No,x1_Yes,x2_No,x2_No internet service,x2_Yes,x3_No,x3_No internet service,x3_Yes,...,x7_Yes,x8_Month-to-month,x8_One year,x8_Two year,x9_No,x9_Yes,x10_Bank transfer (automatic),x10_Credit card (automatic),x10_Electronic check,x10_Mailed check
0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
7039,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
7040,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
7041,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [221]:
# replacing null values
X['TotalCharges'] = X['TotalCharges'].fillna(X['TotalCharges'].mean())

In [222]:
num_data = X.select_dtypes('number')

In [223]:
x = pd.concat([cat_data,num_data], axis=1)

In [224]:
x.shape

(7043, 35)

In [225]:
y = y.factorize()[0]

AttributeError: 'numpy.ndarray' object has no attribute 'factorize'

In [226]:
y

array([0, 0, 1, ..., 0, 1, 0], dtype=int64)

In [227]:
X.columns

Index(['SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges'],
      dtype='object')

In [228]:
x.shape

(7043, 35)

##### Train Test Split

In [229]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [230]:
x_train.shape

(5634, 35)

In [231]:
x_test.shape

(1409, 35)

In [232]:
sm = SMOTEENN()

In [233]:
X_resampled, y_resampled = sm.fit_resample(x,y)

In [234]:
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [235]:
from sklearn.ensemble import RandomForestClassifier

In [236]:
model_rf_smote=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [237]:
model_rf_smote.fit(xr_train,yr_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [238]:
yr_predict1 = model_rf_smote.predict(xr_test)

In [239]:
model_score_r1 = model_rf_smote.score(xr_test, yr_test)

In [240]:
print(model_score_r1)
print(metrics.classification_report(yr_test, yr_predict1))    ##for with smoothen accuracy

0.9251934651762683
              precision    recall  f1-score   support

           0       0.94      0.89      0.92       537
           1       0.91      0.96      0.93       626

    accuracy                           0.93      1163
   macro avg       0.93      0.92      0.92      1163
weighted avg       0.93      0.93      0.92      1163



In [241]:
print(metrics.confusion_matrix(yr_test, yr_predict1))

[[478  59]
 [ 28 598]]


In [242]:
import pickle

In [243]:
filename = 'model.sav'

In [244]:
pickle.dump(model_rf_smote, open(filename, 'wb'))

In [245]:
load_model = pickle.load(open(filename, 'rb'))

In [246]:
model_score_r1 = load_model.score(xr_test, yr_test)

In [247]:
model_score_r1

0.9251934651762683

In [256]:
load_model.predict_proba(xr_test.iloc[:4,:])

array([[0.02936323, 0.97063677],
       [0.4986903 , 0.5013097 ],
       [0.69400378, 0.30599622],
       [0.31537576, 0.68462424]])

In [251]:
yr_test

array([1, 0, 1, ..., 1, 0, 1], dtype=int64)